In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
import sys
sys.path.insert(0, '/home/er647/projects/feature-wise-active-learning')
sys.path.insert(0, '/home/er647/projects/feature-wise-active-learning/src')
sys.path.insert(0, '/home/er647/projects/feature-wise-active-learning/notebooks')

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import wandb
import datetime
from utils import *
from _utils import filter_df_on_conditions, make_subplots
import seaborn as sns
from functools import partial

In [ ]:
datetime.datetime.now(datetime.timezone.utc).isoformat()

In [5]:
api = wandb.Api(timeout=60)

In [6]:
#### Get runs by tag
runs = api.runs('evangeorgerex/fwal', {
	"tags": '264b29f6',
	'State': 'finished'
})

# #### Get one run
# run = api.run('andreimargeloiu/low-data/rwzd9fud')

# #### Get runs by date
# runs = api.runs('andreimargeloiu/low-data', {
# 	"$and": [{
# 		'created_at': {
# 			"$lt": '2022-03-09T10',
# 			"$gt": '2022-03-07T17'
# 		},
# 		'config.embedding_type': 'nmf',
# 	}],
# 	'$or': [
# 		{'tags': '!!!!'},
# 		{'tags': '!!!!'},
# 		{'tags': '!!!!'},
# 		{
# 			"tags": '8e85015a',  # nmf size 50 
# 			'config.gamma': {'$eq': 0}
# 		}
	
# 	],
# 	'State': 'finished',
# })

print(len(runs))
runs = filter_runs(runs, return_dataframe=False)
print(f"Found {len(runs)} runs")
runs_df = transforms_runs_into_dataframe(runs)
print(runs_df.shape)

200
Found 200 runs
(200, 152)


In [ ]:
### Optional commands
# add_tag_and_update(runs, TAG)

# - run.created_at
# - run.config.keys()
# - run.summary._json_dict['bestmodel_train/balanced_accuracy']

In [11]:
runs_df.columns

Index(['name', 'id', '_timestamp', 'bestmodel_test_y_hat',
       'bestmodel_test/balanced_accuracy', 'bestmodel_test/precision_weighted',
       'bestmodel_test/F1_weighted', 'bestmodel_train/cross_entropy_loss',
       '_runtime', 'train/total_loss',
       ...
       'patience_early_stopping', 'concrete_anneal_iterations',
       'cosine_warm_restart_t_mult', 'cosine_warm_restart_eta_min',
       'sparsity_gene_embedding_size', 'sparsity_gene_embedding_type',
       'nonlinearity_weight_predictor', 'evaluate_with_sampled_datasets',
       'sparsity_regularizer_hyperparam', 'layers_for_hidden_representation'],
      dtype='object', length=152)

In [7]:
pd.set_option('display.max_columns', None)
runs_df[
    ['id',
    #  'lr','gamma',
     'mask_type',
     'sparsity_regularizer_hyperparam',
    #  'bestmodel_valid/balanced_accuracy','bestmodel_valid/reconstruction_loss','bestmodel_valid/sparsity_loss',
     'best_mask_parameters']
    ]

,id,mask_type,sparsity_regularizer_hyperparam,best_mask_parameters
0,ikhki7dh,probabilistic,1000.0,"[0.3366183936595917, 0.1287771463394165, 0.234..."
1,aqe3jozq,probabilistic,1000.0,"[0.3365437388420105, 0.12873981893062592, 0.23..."
2,wiy9fpyv,probabilistic,1000.0,"[0.3365899324417114, 0.12876291573047638, 0.23..."
3,ek3m1i82,probabilistic,1000.0,"[0.3365899324417114, 0.12876291573047638, 0.23..."
4,begatpu9,probabilistic,1000.0,"[0.33658725023269653, 0.12876157462596893, 0.2..."
...,...,...,...,...
195,zz8x59c5,sigmoid,0.0,"[0.3403615653514862, 0.24742859601974487, 0.38..."
196,kv4burnd,sigmoid,0.0,"[0.3523191809654236, 0.2648531198501587, 0.382..."
197,99wqap9g,sigmoid,0.0,"[0.37576863169670105, 0.2500225603580475, 0.36..."
198,k5g0jq4i,sigmoid,0.0,"[0.3704529404640198, 0.2280469834804535, 0.353..."


In [8]:
# Splitting the array in 'best_mask_parameters' into individual columns
parameters_df = pd.DataFrame(runs_df['best_mask_parameters'].to_list(), columns=[f'parameter_{i}' for i in range(1, 6)])
parameters_df = pd.concat([runs_df, parameters_df], axis=1)



In [10]:
columns = [f'parameter_{i}' for i in range(1, 6)]
columns.extend(['mask_type','repeat_id','test_split','bestmodel_valid/balanced_accuracy','bestmodel_valid/reconstruction_loss','bestmodel_valid/sparsity_loss'])

parameters_df[parameters_df['sparsity_regularizer_hyperparam']==1000][columns]

,parameter_1,parameter_2,parameter_3,parameter_4,parameter_5,mask_type,repeat_id,test_split,bestmodel_valid/balanced_accuracy,bestmodel_valid/reconstruction_loss,bestmodel_valid/sparsity_loss
0,0.336618,0.128777,0.234407,0.230277,-1.122598,probabilistic,4,4,0.903125,0.223508,2445.169678
1,0.336544,0.128740,0.234369,0.230240,-1.122300,probabilistic,4,3,0.904875,0.177886,2514.360352
2,0.336590,0.128763,0.234392,0.230263,-1.122484,probabilistic,4,2,0.908125,0.174210,2533.942627
3,0.336590,0.128763,0.234392,0.230263,-1.122484,probabilistic,4,1,0.923125,0.178626,2533.942627
4,0.336587,0.128762,0.234391,0.230262,-1.122474,probabilistic,4,0,0.917625,0.176147,2500.000000
20,0.336556,0.128746,0.234375,0.230246,-1.122349,probabilistic,3,4,0.917625,0.185946,2501.305420
21,0.336595,0.128766,0.234395,0.230266,-1.122506,probabilistic,3,3,0.909375,0.186174,2563.968750
22,0.336562,0.128749,0.234378,0.230249,-1.122371,probabilistic,3,2,0.917500,0.188058,2490.861572
23,0.336534,0.128735,0.234365,0.230235,-1.122262,probabilistic,3,1,0.926500,0.159622,2580.939941
24,0.336587,0.128762,0.234391,0.230262,-1.122474,probabilistic,3,0,0.916625,0.178927,2500.000000


In [ ]:
### Plot
pd.options.display.float_format = '{:,.2f}'.format

grouped = runs_df.groupby(['dataset', 'embedding_type'])
g1 = grouped.agg({'bestmodel_train/balanced_accuracy': ['mean', 'std']})*100
g2 = grouped.agg({'bestmodel_test/balanced_accuracy': ['mean', 'std']})*100
res = pd.concat([g1, g2], axis=1).round(decimals=2)
res

In [ ]:
# file_name = 'test'
# res.to_csv(f'/home/er647/projects/feature-wise-active-learning/compute_results/csv/{file_name}.csv')
# res.to_excel(f'/home/er647/projects/feature-wise-active-learning/compute_results/csv/{file_name}.xlsx')